In [14]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

! pip install sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------------------------- 991.5/991.5 kB 15.5 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("Preprocessed_radiology_data.xls")
print(df.head())

          note_id  subject_id     hadm_id note_type  note_seq  \
0  10000032-RR-14    10000032  22595853.0        RR        14   
1  10000032-RR-15    10000032  22595853.0        RR        15   
2  10000032-RR-16    10000032  22595853.0        RR        16   
3  10000032-RR-18    10000032         NaN        RR        18   
4  10000032-RR-20    10000032         NaN        RR        20   

             charttime            storetime   Examination  \
0  2180-05-06 21:19:00  2180-05-06 23:32:00         CHEST   
1  2180-05-06 23:00:00  2180-05-06 23:26:00         LIVER   
2  2180-05-07 09:55:00  2180-05-07 11:15:00           NaN   
3  2180-06-03 12:46:00  2180-06-03 14:01:00    Ultrasound   
4  2180-07-08 13:18:00  2180-07-08 14:15:00  Paracentesis   

                                          Indication  \
0      with new onset ascites  // eval for infection   
1          year-old female with cirrhosis, jaundice.   
2   HCV cirrhosis c/b ascites, hiv on ART, h/o IV...   
3   year old woman

0    CHEST  with new onset ascites  // eval for inf...
1    LIVER  year-old female with cirrhosis, jaundic...
2    nan  HCV cirrhosis c/b ascites, hiv on ART, h/...
3    Ultrasound  year old woman with cirrhosis, asc...
4    Paracentesis  year old woman with cirrhosis, a...
Name: combined_text, dtype: object

In [6]:
df['combined_text'].head()

0    CHEST  with new onset ascites  // eval for inf...
1    LIVER  year-old female with cirrhosis, jaundic...
2    nan  HCV cirrhosis c/b ascites, hiv on ART, h/...
3    Ultrasound  year old woman with cirrhosis, asc...
4    Paracentesis  year old woman with cirrhosis, a...
Name: combined_text, dtype: object

## TF-IDF Summary

In [9]:
def extractive_tfidf_summary(text, num_sentences=3):
    # Split the text into sentences
    sentences = text.split('.')
    
    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sentences)
    
    # Calculate cosine similarity between sentences
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    # Sum the cosine similarity scores for each sentence
    sentence_scores = cosine_sim.sum(axis=1)
    
    # Get indices of the top N sentences
    top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]
    
    # Select and return the top sentences
    summary = '. '.join([sentences[i] for i in top_sentence_indices])
    return summary

# Example: Apply to one of the combined texts
example_text = df['combined_text'][0]
summary_tfidf = extractive_tfidf_summary(example_text)
print(summary_tfidf)


 There is no focal consolidation, pleural effusion or pneumothorax. CHEST  with new onset ascites  // eval for infection Chest PA and lateral None.   Bilateral No acute cardiopulmonary process


In [12]:
def extractive_textrank_summary(text, num_sentences=3):
    # Gensim's summarize function does the sentence ranking automatically
    summary = summarize(text, word_count=100)  # You can adjust this parameter
    return summary

# Example: Apply to one of the combined texts
summary_textrank = extractive_textrank_summary(example_text)
print(summary_textrank)


NameError: name 'summarize' is not defined

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def abstractive_t5_summary(text):
    # Encode the input text and generate the output
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    
    # Generate summary
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    
    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example: Apply to one of the combined texts
summary_t5 = abstractive_t5_summary(example_text)
print(summary_t5)


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
